# 04 - Spatial Validation Deep Dive

## Objectif
Explorer en détail la généralisation spatiale du modèle.
C'est le point critique de ce challenge: le modèle sera évalué sur des sites non vus.

---

## Table des Matières
1. [Contexte et Importance](#1-context)
2. [Analyse de la Distribution Spatiale](#2-spatial-dist)
3. [Stratégies de Validation](#3-strategies)
4. [Diagnostic de Généralisation](#4-diagnosis)
5. [Amélioration de la Robustesse Spatiale](#5-improvement)

---
## 1. Contexte et Importance <a id='1-context'></a>

### Pourquoi la validation spatiale est cruciale?

**Problème**: Les mesures d'un même site partagent des caractéristiques non explicites:
- Géologie locale
- Usages du sol non capturés
- Biais de mesure spécifiques au site

**Conséquence**: Un modèle entraîné avec validation random peut:
- "Mémoriser" les sites plutôt qu'apprendre les relations générales
- Afficher un score CV excellent mais échouer sur de nouveaux sites

**Solution**: Validation Leave-Site-Out ou GroupKFold par site.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score

import sys
sys.path.append('..')
from src.config import TARGETS, RANDOM_STATE
from src.paths import PROCESSED_DATA_DIR, FIGURES_DIR

print("Setup completed.")

---
## 2. Analyse de la Distribution Spatiale <a id='2-spatial-dist'></a>

Comprendre comment les sites sont distribués géographiquement.

In [ ]:
def analyze_site_distribution(df, site_col='site_id', lat_col='latitude', lon_col='longitude'):
    """
    Analyse la distribution des sites.
    """
    # Nombre de sites
    n_sites = df[site_col].nunique()
    print(f"Nombre de sites uniques: {n_sites}")
    
    # Observations par site
    obs_per_site = df.groupby(site_col).size()
    print(f"\nObservations par site:")
    print(obs_per_site.describe())
    
    # Distribution
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram des observations par site
    obs_per_site.hist(bins=30, ax=axes[0], edgecolor='black')
    axes[0].set_xlabel('Nombre d\'observations')
    axes[0].set_ylabel('Nombre de sites')
    axes[0].set_title('Distribution des observations par site')
    
    # Carte des sites
    site_coords = df.groupby(site_col)[[lat_col, lon_col]].first()
    site_coords['n_obs'] = obs_per_site
    
    scatter = axes[1].scatter(
        site_coords[lon_col], site_coords[lat_col],
        c=site_coords['n_obs'], cmap='viridis',
        s=50, alpha=0.7
    )
    plt.colorbar(scatter, ax=axes[1], label='Nombre d\'observations')
    axes[1].set_xlabel('Longitude')
    axes[1].set_ylabel('Latitude')
    axes[1].set_title('Distribution géographique des sites')
    
    plt.tight_layout()
    plt.show()
    
    return obs_per_site

In [ ]:
# Analyse
# obs_per_site = analyze_site_distribution(train)

In [ ]:
def analyze_target_by_site(df, target, site_col='site_id'):
    """
    Analyse la variabilité de la target par site.
    """
    site_stats = df.groupby(site_col)[target].agg(['mean', 'std', 'min', 'max', 'count'])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Distribution des moyennes par site
    site_stats['mean'].hist(bins=30, ax=axes[0], edgecolor='black')
    axes[0].axvline(df[target].mean(), color='red', linestyle='--', label='Global mean')
    axes[0].set_xlabel(f'Mean {target} par site')
    axes[0].set_title('Variabilité inter-sites')
    axes[0].legend()
    
    # Variance intra-site
    site_stats['std'].hist(bins=30, ax=axes[1], edgecolor='black')
    axes[1].set_xlabel(f'Std {target} par site')
    axes[1].set_title('Variabilité intra-site')
    
    plt.tight_layout()
    plt.show()
    
    # Ratio variance inter/intra
    var_inter = site_stats['mean'].var()
    var_intra = (site_stats['std']**2).mean()
    print(f"\nVariance inter-sites: {var_inter:.4f}")
    print(f"Variance intra-site moyenne: {var_intra:.4f}")
    print(f"Ratio inter/intra: {var_inter/var_intra:.2f}")
    print("(Ratio élevé = les sites sont très différents entre eux)")
    
    return site_stats

In [ ]:
# Analyse pour chaque target
# for target in TARGETS:
#     print(f"\n=== {target} ===")
#     analyze_target_by_site(train, target)

---
## 3. Stratégies de Validation <a id='3-strategies'></a>

### Comparaison des approches

In [ ]:
def compare_cv_strategies(model, X, y, groups):
    """
    Compare différentes stratégies de validation croisée.
    
    Logique:
    - Random CV: sous-estime l'erreur de généralisation
    - GroupKFold: estimation réaliste
    - LeaveOneGroupOut: le plus conservateur
    """
    from sklearn.model_selection import KFold
    
    results = {}
    
    # 1. Random 5-fold CV
    print("1. Random 5-fold CV (ATTENTION: trop optimiste!)")
    random_cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    scores = []
    for train_idx, val_idx in random_cv.split(X):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        pred = model.predict(X.iloc[val_idx])
        scores.append(np.sqrt(mean_squared_error(y.iloc[val_idx], pred)))
    results['random'] = {'mean': np.mean(scores), 'std': np.std(scores)}
    print(f"   RMSE: {results['random']['mean']:.4f} ± {results['random']['std']:.4f}")
    
    # 2. Group 5-fold CV
    print("\n2. Spatial GroupKFold (5 folds)")
    group_cv = GroupKFold(n_splits=5)
    scores = []
    for train_idx, val_idx in group_cv.split(X, y, groups):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        pred = model.predict(X.iloc[val_idx])
        scores.append(np.sqrt(mean_squared_error(y.iloc[val_idx], pred)))
    results['group_5fold'] = {'mean': np.mean(scores), 'std': np.std(scores)}
    print(f"   RMSE: {results['group_5fold']['mean']:.4f} ± {results['group_5fold']['std']:.4f}")
    
    # 3. Leave-One-Site-Out (si pas trop de sites)
    n_sites = len(np.unique(groups))
    if n_sites <= 50:  # Limiter pour le temps de calcul
        print(f"\n3. Leave-One-Site-Out ({n_sites} sites)")
        logo_cv = LeaveOneGroupOut()
        scores = []
        for train_idx, val_idx in logo_cv.split(X, y, groups):
            model.fit(X.iloc[train_idx], y.iloc[train_idx])
            pred = model.predict(X.iloc[val_idx])
            scores.append(np.sqrt(mean_squared_error(y.iloc[val_idx], pred)))
        results['logo'] = {'mean': np.mean(scores), 'std': np.std(scores)}
        print(f"   RMSE: {results['logo']['mean']:.4f} ± {results['logo']['std']:.4f}")
    
    # Résumé
    print("\n" + "="*50)
    print("RÉSUMÉ: Écart entre Random et Spatial CV")
    gap = results['group_5fold']['mean'] - results['random']['mean']
    print(f"Écart: {gap:.4f} ({100*gap/results['random']['mean']:.1f}% d'augmentation)")
    print("Un écart important indique un risque d'overfitting spatial!")
    
    return results

In [ ]:
# Comparaison
# from sklearn.ensemble import RandomForestRegressor
# model = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
# cv_comparison = compare_cv_strategies(model, X_train, y_train[TARGETS[0]], site_ids)

---
## 4. Diagnostic de Généralisation <a id='4-diagnosis'></a>

Identifier les sites/régions où le modèle performe mal.

In [ ]:
def diagnose_spatial_performance(y_true, y_pred, site_ids, coords_df=None):
    """
    Diagnostique la performance par site.
    
    Args:
        y_true: Valeurs réelles
        y_pred: Prédictions (OOF)
        site_ids: Identifiants de site
        coords_df: DataFrame avec lat/lon par site (optionnel)
    """
    # Performance par site
    df = pd.DataFrame({
        'y_true': y_true,
        'y_pred': y_pred,
        'residual': y_true - y_pred,
        'site_id': site_ids
    })
    
    site_perf = df.groupby('site_id').apply(lambda x: pd.Series({
        'rmse': np.sqrt(mean_squared_error(x['y_true'], x['y_pred'])),
        'mae': np.abs(x['residual']).mean(),
        'bias': x['residual'].mean(),
        'n_obs': len(x)
    })).reset_index()
    
    # Visualisation
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Distribution du RMSE par site
    site_perf['rmse'].hist(bins=20, ax=axes[0, 0], edgecolor='black')
    axes[0, 0].axvline(site_perf['rmse'].median(), color='red', linestyle='--', label='Median')
    axes[0, 0].set_xlabel('RMSE par site')
    axes[0, 0].set_title('Distribution de la performance par site')
    axes[0, 0].legend()
    
    # 2. Distribution du biais
    site_perf['bias'].hist(bins=20, ax=axes[0, 1], edgecolor='black')
    axes[0, 1].axvline(0, color='red', linestyle='--')
    axes[0, 1].set_xlabel('Biais moyen par site')
    axes[0, 1].set_title('Distribution du biais (positive = sous-estimation)')
    
    # 3. RMSE vs nombre d'observations
    axes[1, 0].scatter(site_perf['n_obs'], site_perf['rmse'], alpha=0.6)
    axes[1, 0].set_xlabel('Nombre d\'observations par site')
    axes[1, 0].set_ylabel('RMSE')
    axes[1, 0].set_title('Performance vs Taille du site')
    
    # 4. Sites problématiques
    top_worst = site_perf.nlargest(10, 'rmse')
    axes[1, 1].barh(range(len(top_worst)), top_worst['rmse'])
    axes[1, 1].set_yticks(range(len(top_worst)))
    axes[1, 1].set_yticklabels(top_worst['site_id'])
    axes[1, 1].set_xlabel('RMSE')
    axes[1, 1].set_title('Top 10 sites les plus difficiles')
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques
    print("\n=== Statistiques de Performance par Site ===")
    print(f"RMSE médian: {site_perf['rmse'].median():.4f}")
    print(f"RMSE moyen: {site_perf['rmse'].mean():.4f}")
    print(f"Écart-type RMSE: {site_perf['rmse'].std():.4f}")
    print(f"\nSites avec biais > 0 (sous-estimation): {(site_perf['bias'] > 0).sum()} / {len(site_perf)}")
    
    return site_perf

In [ ]:
# Diagnostic avec les prédictions OOF
# site_perf = diagnose_spatial_performance(
#     y_train[TARGETS[0]].values,
#     lgb_results[TARGETS[0]]['oof_predictions'],
#     site_ids
# )

---
## 5. Amélioration de la Robustesse Spatiale <a id='5-improvement'></a>

### Stratégies pour améliorer la généralisation

In [ ]:
# Stratégie 1: Features spatiales généralisables
"""
Au lieu d'utiliser des coordonnées brutes, utiliser:
- Altitude, pente, orientation
- Distance à des features géographiques (côte, rivière principale)
- Caractéristiques du bassin versant
- Clusters géographiques (mais attention à la granularité)
"""

# Stratégie 2: Régularisation forte
"""
- Augmenter reg_alpha et reg_lambda
- Réduire la profondeur des arbres
- Augmenter min_child_samples
- Réduire num_leaves
"""

# Stratégie 3: Ensemble de modèles spatiaux
"""
- Entraîner un modèle par région géographique
- Combiner avec un modèle global
- Pondérer selon la distance aux sites d'entraînement
"""

In [ ]:
def spatial_aware_prediction(models, X_train, X_test, train_coords, test_coords, k=5):
    """
    Prédiction pondérée par la proximité spatiale.
    
    Logique:
    - Pour chaque point de test, trouver les k sites d'entraînement les plus proches
    - Pondérer les prédictions des modèles entraînés sur ces sites
    
    Note: Approche expérimentale, à tester.
    """
    from sklearn.neighbors import NearestNeighbors
    
    # Trouver les voisins spatiaux
    nn = NearestNeighbors(n_neighbors=k)
    nn.fit(train_coords)
    distances, indices = nn.kneighbors(test_coords)
    
    # Poids inversement proportionnels à la distance
    weights = 1 / (distances + 1e-6)
    weights = weights / weights.sum(axis=1, keepdims=True)
    
    # Prédictions pondérées
    # (À adapter selon la structure des modèles)
    
    return weights, indices

---
## Conclusions

### Checklist Validation Spatiale

- [ ] Utiliser GroupKFold ou Leave-Site-Out (JAMAIS random CV seul)
- [ ] Comparer les scores random vs spatial pour détecter l'overfitting
- [ ] Analyser la performance par site pour identifier les régions difficiles
- [ ] Vérifier que les features spatiales sont généralisables
- [ ] Ne PAS utiliser site_id ou coordonnées brutes comme features
- [ ] Préférer des features dérivées (altitude, distance, clusters larges)
- [ ] Régulariser suffisamment le modèle